Naive Bayes Classifier Example on anomaly detection: Heart of this classifiers is conditional probability. Three types of models
===========================
1. Multinomial Mode: when features describes descrete frequency count
2. Bernoulli: Binary features
3. Gaussian: when features are normaly distrubuted 

Conditions: Past condition/prior remains same. Predictors are independent 

In [71]:
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
import zipp
import numpy as np

In [59]:

train=pd.read_csv("/home/salam/cnn/ML_Models/Train_data.csv",error_bad_lines=False)
train.head(5)


,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,udp,other,SF,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,tcp,private,S0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,anomaly
3,0,tcp,http,SF,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,tcp,http,SF,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal


In [60]:
test=pd.read_csv("Test_data.csv",error_bad_lines=False)
test.head(5)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0,tcp,private,REJ,0,0,0,0,0,0,...,255,10,0.04,0.06,0.00,0.00,0.0,0.0,1.00,1.00
1,0,tcp,private,REJ,0,0,0,0,0,0,...,255,1,0.00,0.06,0.00,0.00,0.0,0.0,1.00,1.00
2,2,tcp,ftp_data,SF,12983,0,0,0,0,0,...,134,86,0.61,0.04,0.61,0.02,0.0,0.0,0.00,0.00
3,0,icmp,eco_i,SF,20,0,0,0,0,0,...,3,57,1.00,0.00,1.00,0.28,0.0,0.0,0.00,0.00
4,1,tcp,telnet,RSTO,0,15,0,0,0,0,...,29,86,0.31,0.17,0.03,0.02,0.0,0.0,0.83,0.71


In [61]:
#convert non numeric Train column to numeric
#conver non numeric
enc = LabelEncoder()
def convert_non_numeric_to_numeric(data):
    for col in data.columns:
        if str(data.dtypes[col]) is not 'int64':
            enc.fit(data[col])
            data[col] = enc.transform(data[col])
    return data
train_processed=train
train_processed=convert_non_numeric_to_numeric(train_processed)    
train_processed.head()
    

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0,1,19,9,414,0,0,0,0,0,...,25,17,3,17,0,0,0,5,0,1
1,0,2,41,9,126,0,0,0,0,0,...,1,0,60,88,0,0,0,0,0,1
2,0,1,46,5,0,0,0,0,0,0,...,26,10,5,0,0,99,87,0,0,0
3,0,1,22,9,212,2892,0,0,0,0,...,255,100,0,3,4,3,1,0,1,1
4,0,1,22,9,179,362,0,0,0,0,...,255,100,0,0,0,0,0,0,0,1


In [62]:
#convert non numeric Train column to numeric
test_processed=test
test_processed=convert_non_numeric_to_numeric(test_processed)    
test_processed.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0,1,45,1,0,0,0,0,0,0,...,255,10,4,6,0,0,0,0,100,99
1,0,1,45,1,0,0,0,0,0,0,...,255,1,0,6,0,0,0,0,100,99
2,2,1,19,9,986,0,0,0,0,0,...,134,86,61,4,61,2,0,0,0,0
3,0,0,13,9,18,0,0,0,0,0,...,3,57,100,0,100,28,0,0,0,0
4,1,1,55,2,0,7,0,0,0,0,...,29,86,31,17,3,2,0,0,83,71


In [65]:
y_train=train_processed["class"]
x_train=train_processed.drop(["class"], axis=1)
#x_train, x_test,y_train,y_test=train_test_split(X,Y,test_size=0.3,random_state=42)

In [66]:
#train the model
multnomial=MultinomialNB()
multnomial.fit(x_train,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [74]:
y_pred=multnomial.predict(test_processed)
print(np.transpose(y_pred))


[0 0 1 ... 1 1 0]


In [77]:
test["class"]=y_pred
test.to_csv("Predicted_outpu.csv")